In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [1]:
import pandas as pd
import os
import re
import codecs
from IPython.display import display
from six.moves import cPickle as pickle
import string
from PIL import Image
import numpy as np
import h5py

In [2]:
width = None
pd.options.display.max_rows = 600
pd.options.display.max_columns = width
pd.options.display.max_colwidth = 600
pd.options.display.width = width
pd.options.display.max_seq_items = None
pd.options.display.expand_frame_repr = False
pd.options.display.colheader_justify = 'left'

In [3]:
import data_commons as dtc
import dl_commons as dlc
import viz_commons as vc
from viz_commons import VisualizeDir, DiffParams, VisualizeStep

In [4]:
# v = Visualize('./tb_metrics_dev/2017-10-06 17-56-47 PDT/store', '../data/generated2')
# v = VisualizeDir('./tb_metrics/2017-10-08 12-26-45 PDT/store')
# v = VisualizeDir('./tb_metrics_dev/2017-10-09 12-45-15 PDT/store')
# vd = VisualizeDir('./tb_metrics/2017-10-09 17-43-49 PDT/store')
# vd = VisualizeDir('tb_metrics/2017-10-09 16-01-07 PDT/store')
# vd2 = VisualizeDir('tb_metrics/2017-09-26 22-40-18 PDT/new_code 2017-10-10 15-10-17 PDT/store')
# vd = VisualizeDir('./tb_metrics/2017-10-07 14-33-35 PDT_my_decoder/store')
# vd = VisualizeDir('./tb_metrics_view/2017-10-10 19-14-54 PDT good 3_decoder_LSTMs_my_decoder/store')
# vd = VisualizeDir('./tb_metrics_view/2017-10-11 17-46-12 PDT 3lstm_3att/store')
# vd = VisualizeDir('/zpool_3TB/i2l/tb_metrics/2017-10-12 00-15-53 PDT good 3lstm_attMLP/store')
# vd = VisualizeDir('./tb_metrics_view/2017-10-12 19-06-32 PDT 3.1LSTM_noShare_3att/store')
# vd = VisualizeDir('./tb_metrics_view/2017-10-17 15-41-15 PDT 3.1LSTM_3att_noGather/store_2')
# vd = VisualizeDir('./tb_metrics_dev/2017-10-19 00-08-10 PDT reset/store')
# vd = VisualizeDir('./tb_metrics/2017-10-19 22-21-59 PDT reset_3.1LSTM_2init_3out_3att/store')
vd = VisualizeDir('/zpool_3TB/i2l/tb_metrics/2017-10-20 23-45-40 PDT reset_3.1LSTM_2init_3out_3attConv_1beta/validate_154364/store_6')

In [5]:
display(vd.max_steps)
# display(vd2.max_steps)

(154364, 154364)

In [6]:
vs = VisualizeStep(vd, 154364)
vs.strs('validation', 'predicted_ids', 'y', mingle=True)

,predicted_ids / y validation_154364 [/zpool_3TB/i2l/tb_metrics/2017-10-20 23-45-40 PDT reset_3.1LSTM_2init_3out_3attConv_1beta/validate_154364/store_6]
0 (k1),V_b^2=\left (\frac {i^2R^2}{(R^2+1)^2}\right )\eos -1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2
0 (k2),"V_{i}=\left [\frac {ik}{(k,2k')},\frac {(i+1)k}{(k,2k')}\right )\eos \eos \eos \eos \eos \eos \eos \eos"
1 (k1),S_{i}^{(2)}=G_{j}^{(2)}+G_{1}({\bf 2}_{2}+{\cal H}_{2}^{(4)})+S_{0}(R).\eos -1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2
1 (k2),c_2(TX)=c_2(B)+11c_1(B)^2+12\sigma c_1(B).\label {cduetx}\eos \eos \eos \eos \eos \eos
2 (k1),Z(p^{n})=\sum _{n\rightarrow 0}a_{n}^{n}(N)L_{n}(k)\eos \eos \eos -1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2
2 (k2),Z(\beta )=\sum _{\{\sigma \}}exp(-\beta H_{gonihedric}^{3d})\eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos
3 (k1),"g_0(q)=\left (\frac {\xi }{\alpha _0}\right )\partial _1\phi \,\eos \eos \eos \eos \eos \eos \eos -1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2"
3 (k2),\beta _k(t)=\left (\frac {c_0}{2\gamma _0}\right )e^{i\omega t}e^{\gamma _kt}\eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos
4 (k1),"\frac {\nabla ^{2}W}{\nabla ^{2}(\nabla _{0})}=\partial _{\lambda }\psi ^{-1}\partial _{\lambda }\phi |\nabla _{k}\phi ^{-1}\partial _{k}\psi _{2}\,,\eos -1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2-2"
4 (k2),"(\nabla \mathcal {S}_0)^2=-{\hbar ^2\over 4|\psi |^4}(\overline \psi \nabla \psi -\psi \nabla \overline \psi )^2,\label {iug}\eos \eos \eos \eos \eos \eos"
